In [1]:
# 上次课作业简单讲解

In [2]:
# 通用ua
ua = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'

In [3]:
# 天气预报数据抓取
# 入口页面：http://www.weather.com.cn/weather/101020100.shtml
# 抓取最近7天天气预报信息
# 进一步思考：如何从百度搜索入口进入然后获得天气预报信息？

In [4]:
# 17火商品数据抓取
# 入口页面：http://www.17huo.com/list-man-0-50011123-0--2m0.html'
# 抓取商品的url、名称、代码和价格。
# 难点：如何处理分页问题。

In [5]:
# 使用并发提高抓取效率

In [6]:
# 多线程
# 充分利用CPU的数量并发工作。
# Python多线程对计算密集型不友好

In [1]:
import time, threading

# 新线程执行的代码:
def loop():
    print('thread %s is running...' % threading.current_thread().name)
    n = 0
    while n < 5:
        n = n + 1
        print('thread %s >>> %s' % (threading.current_thread().name, n))
        time.sleep(1)
    print('thread %s ended.' % threading.current_thread().name)

print('thread %s is running...' % threading.current_thread().name)
t = threading.Thread(target=loop, name='LoopThread')
t.start()
t.join()
print('thread %s ended.' % threading.current_thread().name)

thread MainThread is running...
thread LoopThread is running...
thread LoopThread >>> 1
thread LoopThread >>> 2
thread LoopThread >>> 3
thread LoopThread >>> 4
thread LoopThread >>> 5
thread LoopThread ended.thread MainThread ended.



In [6]:
# 线程池
# 直接使用线程存在管理难点，比如100个任务 vs 4核CPU
import os
import threadpool

def tfunc(*args, **kws):
    print(args, kws)
    
fargs_1 = ([1, 2], {'one': 1, 'two': 2})
fargs_2 = ([3, 4], {'three': 3, 'four': 4})
fargs = [fargs_1, fargs_2]
pool = threadpool.ThreadPool(os.cpu_count())
threads = threadpool.makeRequests(tfunc, fargs) 
[pool.putRequest(thread) for thread in threads]
pool.wait()

(1, 2)(3, 4) {'four': 4, 'three': 3}
 {'one': 1, 'two': 2}


In [9]:
# 利用线程池获取新浪股票接口数据
import requests

def get_stock_info(code):
    api = 'http://hq.sinajs.cn/list='
    r = requests.get(api + code)
    data_str = r.text.split('=')[1].replace('"', '')[:-2]
    infos = data_str.split(',')
    s = infos[0] + ': '
    s += infos[1] + ', ' # 开
    s += infos[3] + ', ' # 收
    s += infos[4] + ', ' # 高
    s += infos[5] + ', ' # 低
    s += '\n'
    print(s)
    
codes = ['sh601001', 'sz002230', 'sh603858', 'sz002352', 'sz159919', 'sz300678']
pool = threadpool.ThreadPool(os.cpu_count())
threads = threadpool.makeRequests(get_stock_info, codes) 
[pool.putRequest(thread) for thread in threads]
pool.wait()

步长制药: 49.950, 49.810, 50.070, 49.720, 
顺丰控股: 49.900, 50.300, 50.450, 49.820, 
大同煤业: 6.900, 6.970, 7.070, 6.750, 
科大讯飞: 59.810, 60.100, 60.800, 59.510, 




300ETF: 4.691, 4.658, 4.697, 4.658, 

中科信息: 56.160, 54.820, 56.450, 54.100, 



In [10]:
# 异步及其应用
# 异步示意图：https://my.oschina.net/andylucc/blog/906327
# 简单说就是请求发送之后就不管了，该忙什么忙什么，注册一个回调函数，请求完成返回数据之后自动调用这个函数进行处理。
# 优点是在等待数据返回时，本来是不消耗CPU的，但还是等占着CPU资源，因为要等请求返回。现在直接忙别的去了。

In [5]:
# 使用异步线程抓取新浪股票接口数据
from requests_threads import AsyncSession

def fetch_stock_info(session, codes):
    async def fetch_stock_info():
        responses = []
        for code in codes:
            responses.append(await session.get( 'http://hq.sinajs.cn/list=' + code))
        for r in responses:
            data_str = r.text.split('=')[1].replace('"', '')[:-2]
            infos = data_str.split(',')
            s = infos[0] + ': '
            s += infos[1] + ', ' # 开
            s += infos[3] + ', ' # 收
            s += infos[4] + ', ' # 高
            s += infos[5] + ', ' # 低
            s += '\n'
            print(s)
    return fetch_stock_info

session = AsyncSession(n=os.cpu_count())
codes = ['sh601001', 'sz002230', 'sh603858', 'sz002352', 'sz159919', 'sz300678']
f = fetch_stock_info(session, codes)
session.run(f) # 执行完一次后想要再次执行需要重启notebook，这个是twisted库的坑，暂时懒得解决了。

ImportError: No module named 'requests_threads'

In [5]:
# 作业

# 1. 利用www.okex.com提供的API获取币币（vs BTC）的行情信息
#    币列表参考：https://github.com/yingl/bithelper/blob/master/okex_data/coins.py
#    API参考：https://www.okex.com/rest_api.html
# 2. 抓取链家商品房数据，上海二手房前3页，每条记录至少抓取3个字段。